In [ ]:
from google.colab import files
uploaded = files.upload()


Saving train_dataset_train.csv to train_dataset_train.csv


In [2]:
!pip3 install catboost
import pandas as pd
df = pd.read_csv('train_dataset_train.csv')
df = df.drop(columns = ['id', 'entrance_id', 'entrance_nm', 'station_nm', 'line_id', 'line_nm', 'ticket_id'])

df['pass_dttm'] = pd.to_datetime(df['pass_dttm'])
df['hour'] = df['pass_dttm'].dt.hour

df = df.drop(columns = ['pass_dttm'])


df = pd.get_dummies(df, prefix = 'ticket_type', columns = ['ticket_type_nm'])
df = pd.get_dummies(df, prefix = 'station', columns = ['station_id'])


df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,time_to_under,label,hour,ticket_type_1 сутки ЕДИНЫЙ ТК,ticket_type_1 сутки Пригород ТК,ticket_type_3 суток Единый ТК,ticket_type_30 дней,ticket_type_30 дней Пригород,ticket_type_30 дней ЕДИНЫЙ ТК,ticket_type_30 дней Пригород ТК,...,station_81003,station_81004,station_81006,station_81010,station_81011,station_81012,station_81013,station_81014,station_81015,station_81016
0,216.316667,8001,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,648.183333,9011,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,865.333333,7022,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1048.233333,2022,5,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,965.600000,2017,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091016,124.750000,15005,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1091017,56.066667,6007,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1091018,91.283333,2010,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1091019,47.900000,11014,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import catboost

X = df.drop(columns=['time_to_under', 'label'])
y = df['time_to_under']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
                                                    
model = catboost.CatBoostRegressor()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)
label_predict = model.predict(X)

Learning rate set to 0.116865
0:	learn: 220.3852141	total: 178ms	remaining: 2m 58s
1:	learn: 209.8251285	total: 271ms	remaining: 2m 15s
2:	learn: 201.1714467	total: 363ms	remaining: 2m
3:	learn: 194.1120681	total: 456ms	remaining: 1m 53s
4:	learn: 188.1623521	total: 544ms	remaining: 1m 48s
5:	learn: 183.5245788	total: 638ms	remaining: 1m 45s
6:	learn: 179.7456555	total: 732ms	remaining: 1m 43s
7:	learn: 176.5517303	total: 820ms	remaining: 1m 41s
8:	learn: 174.0552267	total: 916ms	remaining: 1m 40s
9:	learn: 171.9866300	total: 1s	remaining: 1m 39s
10:	learn: 170.2899204	total: 1.09s	remaining: 1m 38s
11:	learn: 168.9055121	total: 1.2s	remaining: 1m 38s
12:	learn: 167.7945892	total: 1.29s	remaining: 1m 37s
13:	learn: 166.8310252	total: 1.39s	remaining: 1m 37s
14:	learn: 166.0651351	total: 1.48s	remaining: 1m 37s
15:	learn: 165.4460735	total: 1.59s	remaining: 1m 37s
16:	learn: 164.8945355	total: 1.69s	remaining: 1m 37s
17:	learn: 164.4413138	total: 1.8s	remaining: 1m 38s
18:	learn: 164.07

In [4]:
from sklearn.metrics import r2_score
our_r2 = r2_score(y_test, y_predict)
our_r2

0.5274104976130485

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


X = df.drop(columns=['time_to_under', 'label'])
y = df['label']
X['label'] = label_predict

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
                                                    
model = RandomForestClassifier(n_estimators = 20)
model.fit(X_train, y_train)

y_predict = model.predict(X_test)


In [5]:
from sklearn.metrics import recall_score
our_recall = recall_score(y_test, y_predict, average='micro')
our_recall

0.09995814327221844

In [6]:
0.5*our_recall+0.5*our_r2

0.31461234753736295